In [1]:
import sys,os
sys.path.append("..")

MONGO_URL="localhost:27017"
os.environ["MONGO_URL"] = MONGO_URL

MONGO_INITDB_ROOT_USERNAME="root" # 
os.environ["MONGO_INITDB_ROOT_USERNAME"] = MONGO_INITDB_ROOT_USERNAME

MONGO_INITDB_ROOT_PASSWORD="dr5[gnhn" # 
os.environ["MONGO_INITDB_ROOT_PASSWORD"] = MONGO_INITDB_ROOT_PASSWORD

DATABASE_NAME = "admin"
os.environ["MONGO_INITDB_DATABASE"] = DATABASE_NAME

MONGO_BOT_USERNAME="bot"
os.environ["MONGO_BOT_USERNAME"] = MONGO_BOT_USERNAME

MONGO_BOT_PASSWORD="botpsswd"
os.environ["MONGO_BOT_PASSWORD"] = MONGO_BOT_PASSWORD

MONGO_VENDORBOT_DATABASE = "vendorbot_db"
os.environ["MONGO_VENDORBOT_DATABASE"] = MONGO_VENDORBOT_DATABASE

# root_credentials = dict(
#     DATABASE_NAME = "timerbot_db",
#     DATABASE_HOST = MONGO_URL,
#     DATABASE_ADMIN_USERNAME = MONGO_INITDB_ROOT_USERNAME,
#     DATABASE_ADMIN_PASSWORD = MONGO_INITDB_ROOT_PASSWORD
# )

# database credentials extractor import
from modules.db.dbconfig import root_credentials, vendorbot_credentials, blacklist_rdkit_credentials
from modules.db.mongodb import MongoDriver

db_instances = dict(root=MongoDriver(root_credentials), vendorbot_db=MongoDriver(vendorbot_credentials), blacklist_rdkit_db=MongoDriver(blacklist_rdkit_credentials))
print(db_instances)
# print(root_credentials)
# print(timerbot_credentials)

vendorbot_db_client = db_instances["vendorbot_db"].client
blacklist_rdkit_db_client = db_instances["blacklist_rdkit_db"].client

# db_instance = MongoDriver(vendorbot_credentials)

collection_name = "users_collection"
# user_id = 122267418
# query = {"user_id": user_id}

print(vendorbot_db_client)
vendorbot_db_client.list_database_names()

{'root': <modules.db.mongodb.MongoDriver object at 0x000001F421F27970>, 'vendorbot_db': <modules.db.mongodb.MongoDriver object at 0x000001F421F77430>, 'blacklist_rdkit_db': <modules.db.mongodb.MongoDriver object at 0x000001F423163A00>}
MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin')


['admin', 'blacklist_rdkit_db', 'config', 'local', 'test_db', 'vendorbot_db']

In [2]:
import sys,os
import pandas as pd
sys.path.append("..")
from modules.db.dbschema import UserReagents
from modules.db import dbmodel

# retrieving data from user message
# ищем запись относящуюся к пользователю
user_id = 122267418
mongo_query = {"user_id": user_id}

input_txt_file_path = "../srs/user_reagent_lists_import/radical_subject_reagents.txt"
import_CAS_df = pd.read_csv(input_txt_file_path, header = None)
CAS_list = import_CAS_df[0].tolist()

initial_record = dbmodel.get_records(vendorbot_db_client, db_instances["vendorbot_db"], collection_name, mongo_query)
# test_record2 = {
#     "reagent_requests": [
#                 {
#                     "requested_CAS": "50-00-0"
#                 }
#             ]
#         }

# initial_record
user_reagents_object = UserReagents(**initial_record[0])


In [3]:

# импорт листа реагентов с фильтрациями
user_reagents_object.add_list_of_reagents(user_id, "radical_subject", blacklist_rdkit_db_client, db_instances["blacklist_rdkit_db"], CAS_list)
# экспорт JSON
data = user_reagents_object.export()


Elapsed time: 0.3245 seconds


TypeError: 'MapResult' object is not subscriptable

In [ ]:
# записываем в базу объект 
dbmodel.update_record(vendorbot_db_client, db_instances["vendorbot_db"], collection_name, mongo_query, data)

In [ ]:
data

In [3]:
def update_record(client, db_instance, collection_name: str, query: dict, data: dict):
    db_name = db_instance.DATABASE_NAME
    collection = client[db_name][collection_name]
    # result = collection.insert_one(data)
    result = collection.update(query, {"$set":data}, upsert=True)
    logger.info(f"data upserted into {db_name}, {collection_name}")
    return result

In [10]:
db_name = db_instance.DATABASE_NAME
collection = client[db_name][collection_name]

collection

NameError: name 'db_instance' is not defined

In [11]:
collection.update(mongo_query, {"$set":data}, upsert=True)

NameError: name 'collection' is not defined

In [4]:
from sandbox import resolver
input_txt_file_path = "../srs/user_reagent_lists_import/radical_subject_reagents.txt"
import_CAS_df = pd.read_csv(input_txt_file_path, header = None)
CAS_list = import_CAS_df[0].tolist()

In [5]:
tests = resolver.batch_SMILES_resolve(CAS_list)

Elapsed time: 0.4938 seconds


In [ ]:
tests.get()